In [5]:
import numpy as np
import random
import json

#importing nlp library 
import nltk
from nltk.stem.porter import PorterStemmer

#importing pytorch library
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset


In [6]:
import os
import sys
print(sys.path)
print('**************')
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    print('module path in not available hence appending it to sys.path')
    sys.path.append(module_path)
    print('module appended')

['/path/to/application/app/folder', 'C:\\Users\\Uttam\\Desktop\\Ds\\chatbot_using_pytorch\\notebooks', 'C:\\Users\\Uttam\\anaconda3\\envs\\envpytorch\\python37.zip', 'C:\\Users\\Uttam\\anaconda3\\envs\\envpytorch\\DLLs', 'C:\\Users\\Uttam\\anaconda3\\envs\\envpytorch\\lib', 'C:\\Users\\Uttam\\anaconda3\\envs\\envpytorch', '', 'C:\\Users\\Uttam\\AppData\\Roaming\\Python\\Python37\\site-packages', 'C:\\Users\\Uttam\\anaconda3\\envs\\envpytorch\\lib\\site-packages', 'C:\\Users\\Uttam\\anaconda3\\envs\\envpytorch\\lib\\site-packages\\win32', 'C:\\Users\\Uttam\\anaconda3\\envs\\envpytorch\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Uttam\\anaconda3\\envs\\envpytorch\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Uttam\\anaconda3\\envs\\envpytorch\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Uttam\\.ipython', 'C:\\Users\\Uttam\\Desktop\\Ds\\chatbot_using_pytorch']
**************
C:\Users\Uttam\Desktop\Ds\chatbot_using_pytorch


In [8]:
#importing custom modules
from src.nltk_utils import bag_of_words, tokenize, stem
from src.model import NeuralNet

#### Basic understanding aboot tokenize , stemmimg and bag_of_words

##### Tokenization

In [16]:
sentence = 'How are you john'
print(sentence)
print('tokenised sentence ')
print(tokenize(sentence))

How are you john
tokenised sentence 
['How', 'are', 'you', 'john']


##### Stemming

In [17]:
words = ["organize", "organizes", "organizing"]
print(words)

['organize', 'organizes', 'organizing']


In [18]:
words = [stem(w) for w in words]
print(words)

['organ', 'organ', 'organ']


##### Bag of words

In [20]:
words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
sentence = ["hello", "how", "are", "you"]

In [21]:
#return bag of words array:1 for each known word that exists in the sentence, 0 otherwise
bag_of_words(sentence,words)

array([0., 1., 0., 1., 0., 0., 0.], dtype=float32)

In [27]:
with open(f'C:\\Users\\Uttam\\Desktop\\Ds\\chatbot_using_pytorch\\input\\intents.json', 'r') as f:
    intents = json.load(f)
    print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['what are the payment method available', 'can i pay cash', 'Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with GPay?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Gpay,Paytm,',

In [28]:
all_words = []
tags = []
xy = []

In [29]:
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

In [31]:
tags

['greeting',
 'goodbye',
 'thanks',
 'items',
 'payments',
 'delivery',
 'price',
 'order',
 'funny']

In [32]:
all_words

['Hi',
 'Hey',
 'How',
 'are',
 'you',
 'Is',
 'anyone',
 'there',
 '?',
 'Hello',
 'Good',
 'day',
 'Bye',
 'See',
 'you',
 'later',
 'Goodbye',
 'Thanks',
 'Thank',
 'you',
 'That',
 "'s",
 'helpful',
 'Thank',
 "'s",
 'a',
 'lot',
 '!',
 'Which',
 'items',
 'do',
 'you',
 'have',
 '?',
 'What',
 'kinds',
 'of',
 'items',
 'are',
 'there',
 '?',
 'What',
 'do',
 'you',
 'sell',
 '?',
 'what',
 'are',
 'the',
 'payment',
 'method',
 'available',
 'can',
 'i',
 'pay',
 'cash',
 'Do',
 'you',
 'take',
 'credit',
 'cards',
 '?',
 'Do',
 'you',
 'accept',
 'Mastercard',
 '?',
 'Can',
 'I',
 'pay',
 'with',
 'GPay',
 '?',
 'Are',
 'you',
 'cash',
 'only',
 '?',
 'How',
 'long',
 'does',
 'delivery',
 'take',
 '?',
 'How',
 'long',
 'does',
 'shipping',
 'take',
 '?',
 'When',
 'do',
 'I',
 'get',
 'my',
 'delivery',
 '?',
 'what',
 'is',
 'price',
 'of',
 'tea',
 '?',
 'What',
 'is',
 'price',
 'of',
 'coffee',
 'how',
 'much',
 'i',
 'have',
 'to',
 'pay',
 '?',
 'Please',
 'tell',
 'me',

In [33]:
xy

[(['Hi'], 'greeting'),
 (['Hey'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Thank', "'s", 'a', 'lot', '!'], 'thanks'),
 (['Which', 'items', 'do', 'you', 'have', '?'], 'items'),
 (['What', 'kinds', 'of', 'items', 'are', 'there', '?'], 'items'),
 (['What', 'do', 'you', 'sell', '?'], 'items'),
 (['what', 'are', 'the', 'payment', 'method', 'available'], 'payments'),
 (['can', 'i', 'pay', 'cash'], 'payments'),
 (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'),
 (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'),
 (['Can', 'I', 'pay', 'with', 'GPay', '?'], 'payments'),
 (['Are', 'you', 'cash', 'only', '?'], 'payments'),
 (['How', 'long', 'does', 'delivery', 'take', '?'], 'deliv

In [34]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [35]:
print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

36 patterns
9 tags: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'order', 'payments', 'price', 'thanks']
69 unique stemmed words: ["'s", 'a', 'accept', 'anyon', 'are', 'avail', 'bye', 'can', 'card', 'cash', 'coff', 'coffe', 'credit', 'day', 'deliveri', 'do', 'doe', 'extra', 'funni', 'get', 'give', 'good', 'goodby', 'gpay', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'method', 'much', 'my', 'of', 'onli', 'order', 'pay', 'payment', 'pleas', 'price', 'see', 'sell', 'ship', 'someth', 'sugar', 'take', 'tea', 'tell', 'thank', 'that', 'the', 'there', 'to', 'what', 'when', 'which', 'with', 'you']


In [36]:
# create training data
X_train = []
y_train = []

In [37]:
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

In [38]:
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape, y_train.shape)

(36, 69) (36,)


In [39]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

69 9


In [40]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [41]:
dataset = ChatDataset()
print(dataset)

In [42]:
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)
print(train_loader)

In [43]:
train_set = iter(train_loader)
next(train_set)# this is input, outout  for one batch 

[tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,

In [44]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size)
print(model)

NeuralNet(
  (l1): Linear(in_features=69, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=9, bias=True)
  (relu): ReLU()
)


In [45]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
print(criterion)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(optimizer)

CrossEntropyLoss()
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


In [46]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        labels = labels.type(torch.LongTensor)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.5564
Epoch [200/1000], Loss: 0.1156
Epoch [300/1000], Loss: 0.0073
Epoch [400/1000], Loss: 0.0055
Epoch [500/1000], Loss: 0.0022
Epoch [600/1000], Loss: 0.0013
Epoch [700/1000], Loss: 0.0016
Epoch [800/1000], Loss: 0.0008
Epoch [900/1000], Loss: 0.0004
Epoch [1000/1000], Loss: 0.0003
final loss: 0.0003
